## Import the mock dataset

In [1]:
import os, sys
import pandas as pd
import numpy as np
from pathlib import Path
from tabulate import tabulate



cwd = os.getcwd()
df = pd.read_csv(os.path.join(cwd, 'dataset.csv'))
#change all NaN to none values
df = df.replace({np.nan: None})

#print df in a tabular form
print(tabulate(df, headers = 'keys', tablefmt = 'github'))

|      |    id | APOE   |   sex |   birth0 |   educationt0 |   APOE4 |   wave |   I_age_1 |   hyper |      sys |      dia |   hbd |   totchol |   hypchol |   hchol |   medchol |   diab |   tabldiab |   injecdiab |   v_weight |   lengte |   glucose |   hinfresp |   bypass1 |   alcohol1 |     alcpd |   smk3cat |   ondjr2 | visitdate   |
|------|-------|--------|-------|----------|---------------|---------|--------|-----------|---------|----------|----------|-------|-----------|-----------|---------|-----------|--------|------------|-------------|------------|----------|-----------|------------|-----------|------------|-----------|-----------|----------|-------------|
|    0 | 13109 | E2/E4  |     2 |     1954 |             4 |         |      2 |        65 |       0 | 136.195  |  85      |     2 |      5.5  |         0 |       1 |         2 |      2 |          2 |           2 |       57   |    173   |      5.3  |          2 |         2 |          3 |  2        |         3 |     2011 | 200

In [2]:
df10 = df.iloc[1:11,]
df50 = df.iloc[1:51,]
df100 = df.iloc[1:101,]
df500 = df.iloc[1:501,]
df1000 = df.iloc[1:1001,]

#create csv's
#df10.to_csv("FAIRnotator10.csv")
df50.to_csv("FAIRnotator50.csv")
df100.to_csv("FAIRnotator100.csv")
df500.to_csv("FAIRnotator500.csv")
df1000.to_csv("FAIRnotator1000.csv")

df10

,id,APOE,sex,birth0,educationt0,APOE4,wave,I_age_1,hyper,sys,...,v_weight,lengte,glucose,hinfresp,bypass1,alcohol1,alcpd,smk3cat,ondjr2,visitdate
1,9917,E2/E4,1,1947,3,0,2,57,0,141,...,97.3,163,5.9,2,None,4,None,2,2009,2002-05-02
2,7946,E3/E4,1,1935,6,0,1,57,1,130.442,...,70.5,157.5,5.1,2,1,4,0,3,2003,2009-12-18
3,4564,E3/E4,1,1950,3,1,1,58,0,132,...,80.7,178,3.8,2,2,4,0,3,None,2014-01-26
4,1263,E3/E4,2,1960,2,0,0,60,1,138,...,89.3,162,3.7,2,1,4,0.714286,3,2001,2015-07-03
5,18976,E6/E3,2,1933,7,0,4,67,0,152,...,79,172,6.3,2,1,3,0.142857,3,2018,2006-07-23
6,17212,E2/E3,2,1958,2,0,3,72,0,129,...,78.5,178,6.1,2,1,3,None,2,2013,2005-07-18
7,7356,E6/E3,1,1957,7,0,1,63,1,134.439,...,80.4,164,5.18,1,2,4,0,1,2005,2007-03-30
8,19643,E3/E4,2,1948,4,0,4,67,1,129,...,114.4,158,5.6,2,1,2,1,2,2018,2012-08-24
9,14114,E2/E3,2,1953,4,0,3,62,1,139,...,76.5,163.5,6.9,2,1,4,2,2,2013,2009-09-15
10,10330,E3/E4,2,1959,2,2,2,None,1,120,...,116.6,179,5.4,2,1,4,0.428571,3,2009,2011-01-12


## run triplifier on 10 pt dataframe

In [ ]:

# change path 

f = open("triplier.properties", "w") 
f.write ("""
jdbc.url = jdbc:relique:csv:/C:\Users\msloe\Desktop\FAIRnotatorTest?fileExtension=.csv   
jdbc.user =
jdbc.password =
jdbc.driver = org.relique.jdbc.csv.CsvDriver

repo.type = rdf4j
repo.url = http://172.17.0.2:7200
repo.id = data""")

%%cmd
docker run -d -p 7200:7200 jvsoest/graphdb-free:fairstation

In [ ]:
#have triplifier and graphdb running (https://github.com/MaastrichtU-CDS/triplifier) (jvsoest/graphdb-free:fairstation)
import timeit


start = timeit.timeit()

%%cmd
docker run --rm -p 8080:8080^
   -v %cd%/output.ttl:/output.ttl ^
   -v %cd%/ontology.owl:/ontology.owl ^
   -v %cd%/triplifier.properties:/triplifier.properties ^
   ghcr.io/maastrichtu-cds/triplifier:latest

end = timeit.timeit()
print(end - start)

## Functions for annotation and removal

In [ ]:
import requests

endpoint = "http://localhost:7200/repositories/data/"


def addColumn(ColumnName, ColumnClass):
    query = """
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX dbo: <http://um-cds/ontologies/databaseontology/>
        PREFIX roo: <http://www.cancerdata.org/roo/>
        PREFIX myMapping: <http://localhost/myMapping/>
        PREFIX AlzheimerOntology: <http://scai.fraunhofer.de/AlzheimerOntology#>

        INSERT {
            GRAPH <http://annotation.local/> {

            select ?colUri ?columnName ?tableName 
            where { 
            ?colUri rdfs:subClassOf dbo:ColumnCell;
                dbo:column ?columnName;
                dbo:table ?tableUri.                                    #change
            ?tableUri dbo:table ?tableName.
}
                 owl:equivalentClass ?ColumnClass.
                
            }
        } WHERE {
        BIND(<%s> AS ?ColumnName).
        BIND(<%s> AS ?ColumnClass).
        }

        }
        """ % (ColumnName, ColumnClass)

    annotationResponse = requests.post(endpoint,
    data="update="+query, 
    headers={
        "Content-Type": "application/x-www-form-urlencoded"
    })
    print(annotationResponse.status_code)



def addMapping(localTerm, targetClass, superClass):
    query = """
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX EPND: <http://www.epnd.org#>

        INSERT {
            GRAPH <http://annotation.local/> {
                ?term owl:equivalentClass [
                    rdf:type owl:Class;
                    owl:intersectionOf [
                        rdf:first ?superClass;
                        rdf:rest [
                            rdf:first [
                                rdf:type owl:Class;
                                owl:unionOf [
                                    rdf:first [
                                        rdf:type owl:Restriction;
                                        owl:hasValue ?localValue;
                                        owl:onProperty <http://um-cds/ontologies/databaseontology/has_value>;
                                    ];
                                    rdf:rest rdf:nil;
                                ]
                            ];
                            rdf:rest rdf:nil;
                        ]
                    ]
                ].
            }
        } WHERE { 
            BIND(<%s> AS ?term).
            BIND(<%s> AS ?superClass).
            BIND("%s"^^xsd:string AS ?localValue).
        }
        """ % (targetClass, superClass, localTerm)

    annotationResponse = requests.post(endpoint,
    data="update="+query, 
    headers={
        "Content-Type": "application/x-www-form-urlencoded"
    })
    print(annotationResponse.status_code)


def removeMapping():
    query = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX EPND: <http://www.epnd.org#>
    PREFIX AlzheimerOntology: <http://scai.fraunhofer.de/AlzheimerOntology#>
    
    delete {
        GRAPH <http://mapping.local/> {
            EPND:E2E2 owl:equivalentClass AlzheimerOntology:APOE
                }
            } where { } 

            """ 

    annotationResponse = requests.post(endpoint,
    data="update="+query, 
    headers={
        "Content-Type": "application/x-www-form-urlencoded"
    })
    print(annotationResponse.status_code)

## 10 patient test set

In [ ]:

#APOE column mapping

start1 = timeit.timeit()
addColumn("APOE","AlzheimerOntology:APOE" )
end1 = timeit.timeit()
APOEColumnTime = end1-start1

#APOE value mapping
start2 = timeit.timeit()
addMapping("E2/E2", "EPND:E2E2", "AlzheimerOntology:APOE")
addMapping("E2/E3", "EPND:E2E3", "AlzheimerOntology:APOE")
addMapping("E2/E4", "EPND:E2E4", "AlzheimerOntology:APOE")
addMapping("E3/E3", "EPND:E3E3", "AlzheimerOntology:APOE")
addMapping("E3/E4", "EPND:E3E4", "AlzheimerOntology:APOE")
addMapping("E4/E4", "EPND:E4E4", "AlzheimerOntology:APOE")
end2 = timeit.timeit()
APOEValueTime = end2-start2

#remove annotations
start3 = timeit.timeit()

end3 = timeit.timeit()
APOEremovalTime = end3-start3


print (APOEColumnTime, APOEValueTime, APOEremovalTime)